# README
- 素で書いてみることにより、featuretoolsのありがたみを知る


# expectedの作成

In [1]:
import featuretools
import pandas as pd
import copy


data = featuretools.demo.load_mock_customer()

In [2]:
# いちいちKey打つのだるい&型がわかりにくいので変数化
df_customers = data['customers']
df_sessions = data['sessions']
df_transactions = data['transactions']

In [3]:
entity_set = featuretools.EntitySet(id='data')

entity_set.entity_from_dataframe(entity_id='customer',
                                 dataframe=df_customers,
                                 index='customer_id')

entity_set.entity_from_dataframe(entity_id='session',
                                 dataframe=df_sessions,
                                 index='session_id')

entity_set.entity_from_dataframe(entity_id='transaction',
                                 dataframe=df_transactions,
                                 index='transaction_id')

entity_set

Entityset: data
  Entities:
    customer [Rows: 5, Columns: 4]
    session [Rows: 35, Columns: 4]
    transaction [Rows: 500, Columns: 5]
  Relationships:
    No relationships

In [4]:
relation_customer_and_session = featuretools.Relationship(
    parent_variable=entity_set['customer']['customer_id'],
    child_variable=entity_set['session']['customer_id'])

relation_session_and_transaction = featuretools.Relationship(
    parent_variable=entity_set['session']['session_id'],
    child_variable=entity_set['transaction']['session_id'])

entity_set.add_relationships(relationships=[relation_customer_and_session,
                                            relation_session_and_transaction])

Entityset: data
  Entities:
    customer [Rows: 5, Columns: 4]
    session [Rows: 35, Columns: 4]
    transaction [Rows: 500, Columns: 5]
  Relationships:
    session.customer_id -> customer.customer_id
    transaction.session_id -> session.session_id

In [5]:
agg_numeric = ['sum', 'max',
               'count', 'mode'
               ]
agg_trans = ['year', 'month']  # いろいろ変えてみるといいよ

expected, feature_defs = featuretools.dfs(entityset=entity_set,
                                          target_entity='session',
                                          agg_primitives=agg_numeric,
                                          trans_primitives=agg_trans,
                                          max_depth=1)

In [6]:
expected.head(3).T

session_id,1,2,3
customer_id,2,5,4
device,desktop,mobile,mobile
SUM(transaction.amount),1229.01,746.96,1329
MAX(transaction.amount),141.66,135.25,147.73
COUNT(transaction),16,10,15
MODE(transaction.product_id),3,5,1
YEAR(session_start),2014,2014,2014
MONTH(session_start),1,1,1
customer.zip_code,13244,60091,60091


In [7]:
expected.head(3)

,customer_id,device,SUM(transaction.amount),MAX(transaction.amount),COUNT(transaction),MODE(transaction.product_id),YEAR(session_start),MONTH(session_start),customer.zip_code
session_id,,,,,,,,,
1,2,desktop,1229.01,141.66,16,3,2014,1,13244
2,5,mobile,746.96,135.25,10,5,2014,1,60091
3,4,mobile,1329.00,147.73,15,1,2014,1,60091


# ここから自作する ※未完

## 関係ないカラムを除去しておく

In [24]:
transactions = df_transactions.drop(columns=['transaction_time']).copy()

transactions.head()

,transaction_id,session_id,product_id,amount
0,298,1,5,127.64
1,2,1,2,109.48
2,308,1,3,95.06
3,116,1,4,78.92
4,371,1,3,31.54


In [25]:
sessions = df_sessions.copy()

sessions.head()

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30


## Numericの部分だけのDFを作る

In [26]:
merged = pd.merge(sessions,
                  transactions,
                  on='session_id',
                  how='left')

merged.head()

,session_id,customer_id,device,session_start,transaction_id,product_id,amount
0,1,2,desktop,2014-01-01,298,5,127.64
1,1,2,desktop,2014-01-01,2,2,109.48
2,1,2,desktop,2014-01-01,308,3,95.06
3,1,2,desktop,2014-01-01,116,4,78.92
4,1,2,desktop,2014-01-01,371,3,31.54


In [66]:
agg_numeric = merged.groupby(['session_id', 'transaction_id'], as_index=False).agg({'amount': [sum, max]})

agg_numeric.head()

session_id transaction_id  amount        
                                sum     max
0          1              2  109.48  109.48
1          1             10   57.39   57.39
2          1             29   20.91   20.91
3          1            116   78.92   78.92
4          1            158  141.66  141.66

In [67]:
new_col_names = ['session_id']

for level_0 in agg_numeric.columns.levels[0]:
    if level_0 == 'session_id':
        continue

    for col_agg in agg_numeric.columns.levels[1][:-1]:
        new_col_name = f'{level_0.upper()}_{col_agg}'
        new_col_names.append(new_col_name)


agg_numeric.columns = new_col_names

ValueError: Length mismatch: Expected axis has 4 elements, new values have 5 elements

In [ ]:
agg_numeric.head()

## Countの部分だけをつくる

In [68]:
expected.head()

,customer_id,device,SUM(transaction.amount),MAX(transaction.amount),COUNT(transaction),MODE(transaction.product_id),YEAR(session_start),MONTH(session_start),customer.zip_code
session_id,,,,,,,,,
1,2,desktop,1229.01,141.66,16,3,2014,1,13244
2,5,mobile,746.96,135.25,10,5,2014,1,60091
3,4,mobile,1329.00,147.73,15,1,2014,1,60091
4,1,mobile,1613.93,129.00,25,5,2014,1,60091
5,4,mobile,777.02,139.20,11,5,2014,1,60091


In [69]:
agg_count = merged.groupby('session_id', as_index=False).agg(
    {'transaction_id': [pd.Series.count]})

agg_count.columns = ['session_id', 'TRANSACTION_count']

agg_count.head()

,session_id,TRANSACTION_count
0,1,16
1,2,10
2,3,15
3,4,25
4,5,11


## ここまでをマージしてみる


In [55]:
tmp = pd.merge(agg_numeric, agg_count, on='session_id', how='left')

tmp.head()

,session_id,AMOUNT_sum,AMOUNT_max,TRANSACTION_count
0,1,1229.01,141.66,16
1,2,746.96,135.25,10
2,3,1329.00,147.73,15
3,4,1613.93,129.00,25
4,5,777.02,139.20,11


In [57]:
# 比較
expected.head().reset_index()

,session_id,customer_id,device,SUM(transaction.amount),MAX(transaction.amount),COUNT(transaction),MODE(transaction.product_id),YEAR(session_start),MONTH(session_start),customer.zip_code
0,1,2,desktop,1229.01,141.66,16,3,2014,1,13244
1,2,5,mobile,746.96,135.25,10,5,2014,1,60091
2,3,4,mobile,1329.00,147.73,15,1,2014,1,60091
3,4,1,mobile,1613.93,129.00,25,5,2014,1,60091
4,5,4,mobile,777.02,139.20,11,5,2014,1,60091


In [62]:
merged.groupby(by=['session_id', 'customer_id']).count()

,,device,session_start,transaction_id,product_id,amount
session_id,customer_id,,,,,
1,2,16,16,16,16,16
2,5,10,10,10,10,10
3,4,15,15,15,15,15
4,1,25,25,25,25,25
5,4,11,11,11,11,11
6,1,15,15,15,15,15
7,3,15,15,15,15,15
8,4,18,18,18,18,18
9,1,15,15,15,15,15
